In [2]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

In [5]:
from keras.layers import Input, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Conv2D, BatchNormalization, Activation, Concatenate
from keras.models import Model
from keras.datasets import cifar10

## DenseNet

In [ ]:
def dense_block(x, blocks, name, growth_rate=32):
    for i in range(blocks):
        x = conv_block(growth_rate, name=name+'_block'+str(i+1))(x)
    return x

def transition_block(x, reduction, name):
    x = BatchNormalization(epsilon=1.001e-5, name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False,
            name=name + '_conv')(x)
    x = AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    x1 = BatchNormalization(epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False,
            name=name + '_1_conv')(x1)
    x1 = BatchNormalization(epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False,
               name=name + '_2_conv')(x1)
    x = Concatenate(name=name + '_concat')([x, x1])
    return x

In [12]:
def DenseNet(depth, growth_rate, input_shape, classes=10):
    # No BC -> divide by 3
    # BC -> divide by 6
    n_layers = (depth - 4) // 6

    img_input = Input(shape=input_shape)

    x = ZeroPadding2D(img_input, padding=(1, 1))
    x = Conv2D(x, growth_rate*2, 3, use_bias=False, name='conv1/conv')

    x = dense_block(x, n_layers, 'conv2', growth_rate)
    x = transition_block(x, 0.5, 'pool2')
    x = dense_block(x, n_layers, 'conv3', growth_rate)
    x = transition_block(x, 0.5, 'pool3')
    x = dense_block(x, n_layers, 'conv4', growth_rate)

    x = BatchNormalization(epsilon=1.001e-5, name='bn')(x)
    x = Activation('relu', name='relu')(x)
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(classes, name='output')(x)

    return Model(img_input, x)

## Train

In [7]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)